# Imports and setup

In [4]:
!pip install -r requirements.txt

253.36s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 8.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.0
    Uninstalling numpy-1.24.0:
      Successfully uninstalled numpy-1.24.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 4.38.1 requires urllib3~=2.0, but you have urllib3 1.26.20 which is incompatible.


In [5]:
import os
import torch
import monai
from tqdm import tqdm
from statistics import mean
from torch.utils.data import DataLoader
# from torchvision import datasets, transforms
from torch.optim import Adam
import torch.nn as nn
from torch.nn.functional import threshold, normalize
import src.utils as utils

from src.brats_dataset import BratsDataset, collate_fn
from src.brats_processor import Samprocessor, find_slices


from src.segment_anything import build_sam_vit_b, SamPredictor
from src.lora import LoRA_sam
import matplotlib.pyplot as plt
import yaml
import torch.nn.functional as F

print("Setup complete")

Setup complete
